# 07. 지속적 모니터링을 위한 스케줄링 시스템

**담당**: Claude Opus  
**작성일**: 2025-10-24  
**목적**: 자동화된 주기적 드론 작물 모니터링을 위한 스케줄링 시스템

## 주요 기능
- 크론 기반 작업 스케줄링
- 실시간 모니터링 대시보드
- 작업 큐 관리
- 알림 및 경고 시스템
- 작업 실행 이력 관리

In [ ]:
# 필요 패키지 설치
import sys
!{sys.executable} -m pip install schedule apscheduler watchdog pyyaml

In [ ]:
import os
import time
import json
import yaml
import threading
import queue
import logging
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Callable, Any, Union
from dataclasses import dataclass, field, asdict
from enum import Enum
import schedule
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.cron import CronTrigger
from apscheduler.triggers.interval import IntervalTrigger
from apscheduler.triggers.date import DateTrigger
from apscheduler.executors.pool import ThreadPoolExecutor, ProcessPoolExecutor
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import sqlite3
import pickle
from collections import defaultdict, deque
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import uuid

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

## 1. 작업 정의 클래스

In [ ]:
class JobStatus(Enum):
    """작업 상태"""
    PENDING = "pending"
    RUNNING = "running"
    COMPLETED = "completed"
    FAILED = "failed"
    CANCELLED = "cancelled"
    SCHEDULED = "scheduled"

class JobType(Enum):
    """작업 유형"""
    IMAGE_PROCESSING = "image_processing"
    VIDEO_ANALYSIS = "video_analysis"
    BATCH_DETECTION = "batch_detection"
    DATABASE_BACKUP = "database_backup"
    REPORT_GENERATION = "report_generation"
    FOLDER_MONITORING = "folder_monitoring"
    DATA_CLEANUP = "data_cleanup"

@dataclass
class Job:
    """작업 정의"""
    id: str = field(default_factory=lambda: str(uuid.uuid4()))
    name: str = ""
    job_type: JobType = JobType.IMAGE_PROCESSING
    function: Optional[Callable] = None
    args: tuple = field(default_factory=tuple)
    kwargs: dict = field(default_factory=dict)
    schedule_expression: Optional[str] = None  # 크론 표현식
    interval_seconds: Optional[int] = None     # 간격 (초)
    run_at: Optional[datetime] = None         # 특정 시간
    max_retries: int = 3
    retry_delay: int = 60  # 초
    timeout: Optional[int] = None  # 초
    priority: int = 5  # 1-10 (1이 가장 높음)
    enabled: bool = True
    metadata: Dict = field(default_factory=dict)
    
    def __lt__(self, other):
        return self.priority < other.priority

@dataclass
class JobResult:
    """작업 실행 결과"""
    job_id: str
    job_name: str
    status: JobStatus
    started_at: datetime
    completed_at: Optional[datetime] = None
    duration: Optional[float] = None
    result: Any = None
    error: Optional[str] = None
    retry_count: int = 0
    
    def to_dict(self) -> Dict:
        return {
            'job_id': self.job_id,
            'job_name': self.job_name,
            'status': self.status.value,
            'started_at': self.started_at.isoformat(),
            'completed_at': self.completed_at.isoformat() if self.completed_at else None,
            'duration': self.duration,
            'error': self.error,
            'retry_count': self.retry_count
        }

## 2. 작업 실행 이력 관리

In [ ]:
class JobHistoryManager:
    """작업 실행 이력 관리"""
    
    def __init__(self, db_path: str = "job_history.db"):
        self.db_path = db_path
        self._init_database()
        
    def _init_database(self):
        """데이터베이스 초기화"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS job_history (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                job_id TEXT NOT NULL,
                job_name TEXT NOT NULL,
                job_type TEXT,
                status TEXT NOT NULL,
                started_at TIMESTAMP NOT NULL,
                completed_at TIMESTAMP,
                duration REAL,
                error TEXT,
                retry_count INTEGER DEFAULT 0,
                metadata TEXT
            )
        ''')
        
        cursor.execute('''
            CREATE INDEX IF NOT EXISTS idx_job_id ON job_history(job_id);
        ''')
        
        cursor.execute('''
            CREATE INDEX IF NOT EXISTS idx_started_at ON job_history(started_at);
        ''')
        
        conn.commit()
        conn.close()
        
    def save_result(self, result: JobResult, job_type: Optional[JobType] = None):
        """작업 결과 저장"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
            INSERT INTO job_history 
            (job_id, job_name, job_type, status, started_at, completed_at, duration, error, retry_count)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            result.job_id,
            result.job_name,
            job_type.value if job_type else None,
            result.status.value,
            result.started_at,
            result.completed_at,
            result.duration,
            result.error,
            result.retry_count
        ))
        
        conn.commit()
        conn.close()
        
    def get_history(self, 
                   job_id: Optional[str] = None,
                   status: Optional[JobStatus] = None,
                   date_from: Optional[datetime] = None,
                   date_to: Optional[datetime] = None,
                   limit: int = 100) -> List[Dict]:
        """작업 이력 조회"""
        conn = sqlite3.connect(self.db_path)
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        query = "SELECT * FROM job_history WHERE 1=1"
        params = []
        
        if job_id:
            query += " AND job_id = ?"
            params.append(job_id)
            
        if status:
            query += " AND status = ?"
            params.append(status.value)
            
        if date_from:
            query += " AND started_at >= ?"
            params.append(date_from)
            
        if date_to:
            query += " AND started_at <= ?"
            params.append(date_to)
            
        query += " ORDER BY started_at DESC LIMIT ?"
        params.append(limit)
        
        cursor.execute(query, params)
        results = [dict(row) for row in cursor.fetchall()]
        
        conn.close()
        return results
    
    def get_statistics(self, days: int = 7) -> Dict:
        """통계 정보 조회"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        date_from = datetime.now() - timedelta(days=days)
        
        # 전체 통계
        cursor.execute('''
            SELECT 
                COUNT(*) as total,
                SUM(CASE WHEN status = 'completed' THEN 1 ELSE 0 END) as completed,
                SUM(CASE WHEN status = 'failed' THEN 1 ELSE 0 END) as failed,
                AVG(duration) as avg_duration
            FROM job_history
            WHERE started_at >= ?
        ''', (date_from,))
        
        stats = dict(cursor.fetchone())
        
        # 작업 유형별 통계
        cursor.execute('''
            SELECT job_type, COUNT(*) as count, AVG(duration) as avg_duration
            FROM job_history
            WHERE started_at >= ? AND job_type IS NOT NULL
            GROUP BY job_type
        ''', (date_from,))
        
        stats['by_type'] = {row[0]: {'count': row[1], 'avg_duration': row[2]} 
                           for row in cursor.fetchall()}
        
        conn.close()
        return stats
    
    def cleanup_old_records(self, days: int = 30):
        """오래된 기록 정리"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cutoff_date = datetime.now() - timedelta(days=days)
        
        cursor.execute('''
            DELETE FROM job_history WHERE started_at < ?
        ''', (cutoff_date,))
        
        deleted_count = cursor.rowcount
        conn.commit()
        conn.close()
        
        logger.info(f"오래된 기록 {deleted_count}개 삭제")
        return deleted_count

## 3. 폴더 모니터링

In [ ]:
class FolderMonitor(FileSystemEventHandler):
    """폴더 변경 사항 모니터링"""
    
    def __init__(self, callback: Callable, file_extensions: List[str] = None):
        self.callback = callback
        self.file_extensions = file_extensions or ['.jpg', '.jpeg', '.png', '.mp4', '.avi']
        self.processing_queue = queue.Queue()
        self.processed_files = set()
        
    def on_created(self, event):
        """파일 생성 이벤트"""
        if not event.is_directory:
            file_path = Path(event.src_path)
            if file_path.suffix.lower() in self.file_extensions:
                if str(file_path) not in self.processed_files:
                    logger.info(f"새 파일 감지: {file_path}")
                    self.processing_queue.put(str(file_path))
                    self.processed_files.add(str(file_path))
                    
                    # 콜백 실행
                    if self.callback:
                        try:
                            self.callback(str(file_path))
                        except Exception as e:
                            logger.error(f"콜백 실행 실패: {e}")
    
    def on_modified(self, event):
        """파일 수정 이벤트"""
        # 필요시 처리
        pass
    
    def get_pending_files(self) -> List[str]:
        """대기 중인 파일 목록"""
        files = []
        while not self.processing_queue.empty():
            try:
                files.append(self.processing_queue.get_nowait())
            except queue.Empty:
                break
        return files

class FolderWatcher:
    """폴더 감시자"""
    
    def __init__(self):
        self.observers = {}
        self.monitors = {}
        
    def add_watch(self, folder_path: str, callback: Callable, 
                 file_extensions: List[str] = None, recursive: bool = True) -> str:
        """폴더 감시 추가"""
        watch_id = str(uuid.uuid4())
        
        # 모니터 생성
        monitor = FolderMonitor(callback, file_extensions)
        self.monitors[watch_id] = monitor
        
        # Observer 생성
        observer = Observer()
        observer.schedule(monitor, folder_path, recursive=recursive)
        observer.start()
        self.observers[watch_id] = observer
        
        logger.info(f"폴더 감시 시작: {folder_path} (ID: {watch_id})")
        return watch_id
    
    def remove_watch(self, watch_id: str):
        """폴더 감시 제거"""
        if watch_id in self.observers:
            self.observers[watch_id].stop()
            self.observers[watch_id].join()
            del self.observers[watch_id]
            del self.monitors[watch_id]
            logger.info(f"폴더 감시 중지: {watch_id}")
    
    def stop_all(self):
        """모든 감시 중지"""
        for watch_id in list(self.observers.keys()):
            self.remove_watch(watch_id)

## 4. 알림 시스템

In [ ]:
class AlertLevel(Enum):
    """알림 레벨"""
    INFO = "info"
    WARNING = "warning"
    ERROR = "error"
    CRITICAL = "critical"

@dataclass
class Alert:
    """알림 메시지"""
    level: AlertLevel
    title: str
    message: str
    timestamp: datetime = field(default_factory=datetime.now)
    metadata: Dict = field(default_factory=dict)

class NotificationService:
    """알림 서비스"""
    
    def __init__(self):
        self.handlers = []
        self.alert_history = deque(maxlen=1000)
        
    def add_handler(self, handler: Callable):
        """알림 핸들러 추가"""
        self.handlers.append(handler)
        
    def send_alert(self, alert: Alert):
        """알림 전송"""
        self.alert_history.append(alert)
        
        for handler in self.handlers:
            try:
                handler(alert)
            except Exception as e:
                logger.error(f"알림 핸들러 실행 실패: {e}")
    
    def get_recent_alerts(self, count: int = 10, level: Optional[AlertLevel] = None) -> List[Alert]:
        """최근 알림 조회"""
        alerts = list(self.alert_history)
        
        if level:
            alerts = [a for a in alerts if a.level == level]
        
        return alerts[-count:]

class EmailNotifier:
    """이메일 알림 전송"""
    
    def __init__(self, smtp_config: Dict):
        self.smtp_server = smtp_config.get('server', 'localhost')
        self.smtp_port = smtp_config.get('port', 587)
        self.username = smtp_config.get('username')
        self.password = smtp_config.get('password')
        self.from_email = smtp_config.get('from_email')
        self.to_emails = smtp_config.get('to_emails', [])
        
    def send(self, alert: Alert):
        """이메일 전송"""
        if alert.level in [AlertLevel.ERROR, AlertLevel.CRITICAL]:
            try:
                msg = MIMEMultipart()
                msg['From'] = self.from_email
                msg['To'] = ', '.join(self.to_emails)
                msg['Subject'] = f"[{alert.level.value.upper()}] {alert.title}"
                
                body = f"""
                알림 레벨: {alert.level.value}
                시간: {alert.timestamp}
                
                {alert.message}
                
                추가 정보:
                {json.dumps(alert.metadata, indent=2)}
                """
                
                msg.attach(MIMEText(body, 'plain'))
                
                # SMTP 연결 및 전송 (실제 구현시)
                # with smtplib.SMTP(self.smtp_server, self.smtp_port) as server:
                #     server.starttls()
                #     server.login(self.username, self.password)
                #     server.send_message(msg)
                
                logger.info(f"이메일 알림 전송: {alert.title}")
                
            except Exception as e:
                logger.error(f"이메일 전송 실패: {e}")

class ConsoleNotifier:
    """콘솔 알림 출력"""
    
    def send(self, alert: Alert):
        """콘솔 출력"""
        colors = {
            AlertLevel.INFO: '\033[94m',     # 파란색
            AlertLevel.WARNING: '\033[93m',   # 노란색
            AlertLevel.ERROR: '\033[91m',     # 빨간색
            AlertLevel.CRITICAL: '\033[95m'   # 자주색
        }
        
        color = colors.get(alert.level, '')
        reset = '\033[0m'
        
        print(f"{color}[{alert.level.value.upper()}] {alert.timestamp} - {alert.title}{reset}")
        print(f"  {alert.message}")
        
        if alert.metadata:
            print(f"  메타데이터: {alert.metadata}")

## 5. 메인 스케줄러

In [ ]:
class SchedulingSystem:
    """통합 스케줄링 시스템"""
    
    def __init__(self, config_file: Optional[str] = None):
        # 스케줄러 초기화
        self.scheduler = BackgroundScheduler(
            executors={
                'default': ThreadPoolExecutor(20),
                'processpool': ProcessPoolExecutor(5)
            },
            job_defaults={
                'coalesce': False,
                'max_instances': 3
            }
        )
        
        # 컴포넌트 초기화
        self.history_manager = JobHistoryManager()
        self.folder_watcher = FolderWatcher()
        self.notification_service = NotificationService()
        
        # 알림 핸들러 추가
        console_notifier = ConsoleNotifier()
        self.notification_service.add_handler(console_notifier.send)
        
        # 작업 레지스트리
        self.registered_jobs = {}
        
        # 설정 로드
        if config_file:
            self.load_config(config_file)
        
        # 스케줄러 시작
        self.scheduler.start()
        logger.info("스케줄링 시스템 시작")
    
    def register_job(self, job: Job) -> str:
        """작업 등록"""
        if not job.enabled:
            logger.info(f"작업 비활성화: {job.name}")
            return job.id
        
        # APScheduler 작업 추가
        if job.schedule_expression:  # 크론 표현식
            trigger = CronTrigger.from_crontab(job.schedule_expression)
        elif job.interval_seconds:  # 간격
            trigger = IntervalTrigger(seconds=job.interval_seconds)
        elif job.run_at:  # 특정 시간
            trigger = DateTrigger(run_date=job.run_at)
        else:
            logger.error(f"작업 스케줄 정의 없음: {job.name}")
            return job.id
        
        # 작업 래퍼 함수
        def job_wrapper():
            self._execute_job(job)
        
        # 스케줄러에 추가
        self.scheduler.add_job(
            job_wrapper,
            trigger=trigger,
            id=job.id,
            name=job.name,
            replace_existing=True
        )
        
        # 레지스트리에 저장
        self.registered_jobs[job.id] = job
        
        logger.info(f"작업 등록: {job.name} (ID: {job.id})")
        return job.id
    
    def _execute_job(self, job: Job):
        """작업 실행"""
        result = JobResult(
            job_id=job.id,
            job_name=job.name,
            status=JobStatus.RUNNING,
            started_at=datetime.now()
        )
        
        logger.info(f"작업 시작: {job.name}")
        
        try:
            # 타임아웃 처리
            if job.timeout:
                import signal
                
                def timeout_handler(signum, frame):
                    raise TimeoutError(f"작업 타임아웃: {job.timeout}초")
                
                # signal.signal(signal.SIGALRM, timeout_handler)
                # signal.alarm(job.timeout)
            
            # 작업 실행
            if job.function:
                result.result = job.function(*job.args, **job.kwargs)
            
            # 성공
            result.status = JobStatus.COMPLETED
            result.completed_at = datetime.now()
            result.duration = (result.completed_at - result.started_at).total_seconds()
            
            logger.info(f"작업 완료: {job.name} (소요시간: {result.duration:.2f}초)")
            
            # 성공 알림
            if job.job_type in [JobType.DATABASE_BACKUP, JobType.REPORT_GENERATION]:
                self.notification_service.send_alert(Alert(
                    level=AlertLevel.INFO,
                    title=f"작업 완료: {job.name}",
                    message=f"작업이 성공적으로 완료되었습니다. (소요시간: {result.duration:.2f}초)"
                ))
            
        except Exception as e:
            # 실패
            result.status = JobStatus.FAILED
            result.error = str(e)
            result.completed_at = datetime.now()
            result.duration = (result.completed_at - result.started_at).total_seconds()
            
            logger.error(f"작업 실패: {job.name} - {e}")
            
            # 실패 알림
            self.notification_service.send_alert(Alert(
                level=AlertLevel.ERROR,
                title=f"작업 실패: {job.name}",
                message=str(e),
                metadata={'job_id': job.id, 'retry_count': result.retry_count}
            ))
            
            # 재시도
            if result.retry_count < job.max_retries:
                result.retry_count += 1
                logger.info(f"작업 재시도 예정: {job.name} ({result.retry_count}/{job.max_retries})")
                
                # 재시도 스케줄
                retry_time = datetime.now() + timedelta(seconds=job.retry_delay)
                self.scheduler.add_job(
                    lambda: self._execute_job(job),
                    trigger=DateTrigger(run_date=retry_time),
                    id=f"{job.id}_retry_{result.retry_count}"
                )
        
        finally:
            # 타임아웃 해제
            if job.timeout:
                pass  # signal.alarm(0)
            
            # 이력 저장
            self.history_manager.save_result(result, job.job_type)
    
    def remove_job(self, job_id: str):
        """작업 제거"""
        try:
            self.scheduler.remove_job(job_id)
            if job_id in self.registered_jobs:
                del self.registered_jobs[job_id]
            logger.info(f"작업 제거: {job_id}")
        except Exception as e:
            logger.error(f"작업 제거 실패: {e}")
    
    def pause_job(self, job_id: str):
        """작업 일시정지"""
        self.scheduler.pause_job(job_id)
        logger.info(f"작업 일시정지: {job_id}")
    
    def resume_job(self, job_id: str):
        """작업 재개"""
        self.scheduler.resume_job(job_id)
        logger.info(f"작업 재개: {job_id}")
    
    def get_status(self) -> Dict:
        """시스템 상태 조회"""
        jobs = self.scheduler.get_jobs()
        
        return {
            'scheduler_running': self.scheduler.running,
            'total_jobs': len(jobs),
            'active_jobs': len([j for j in jobs if not j.pending]),
            'pending_jobs': len([j for j in jobs if j.pending]),
            'watched_folders': len(self.folder_watcher.observers),
            'recent_alerts': len(self.notification_service.alert_history),
            'job_statistics': self.history_manager.get_statistics()
        }
    
    def load_config(self, config_file: str):
        """설정 파일 로드"""
        with open(config_file, 'r') as f:
            config = yaml.safe_load(f)
        
        # 작업 설정 로드
        for job_config in config.get('jobs', []):
            job = Job(
                name=job_config['name'],
                job_type=JobType(job_config.get('type', 'image_processing')),
                schedule_expression=job_config.get('schedule'),
                interval_seconds=job_config.get('interval'),
                enabled=job_config.get('enabled', True)
            )
            self.register_job(job)
        
        # 폴더 감시 설정
        for watch_config in config.get('folder_watches', []):
            self.folder_watcher.add_watch(
                watch_config['path'],
                lambda x: logger.info(f"새 파일: {x}"),
                watch_config.get('extensions'),
                watch_config.get('recursive', True)
            )
    
    def shutdown(self):
        """시스템 종료"""
        logger.info("스케줄링 시스템 종료 중...")
        
        # 스케줄러 종료
        self.scheduler.shutdown(wait=True)
        
        # 폴더 감시 종료
        self.folder_watcher.stop_all()
        
        logger.info("스케줄링 시스템 종료 완료")

## 6. 사용 예제

In [ ]:
# 예제 작업 함수들
def process_drone_images():
    """드론 이미지 처리 작업"""
    logger.info("드론 이미지 처리 시작")
    time.sleep(2)  # 처리 시뮬레이션
    return {"processed": 10, "failed": 0}

def backup_database():
    """데이터베이스 백업 작업"""
    logger.info("데이터베이스 백업 시작")
    time.sleep(1)
    return {"backup_file": "backup_20251024.db", "size_mb": 156}

def generate_daily_report():
    """일일 리포트 생성"""
    logger.info("일일 리포트 생성 시작")
    time.sleep(1)
    return {"report_file": "daily_report_20251024.pdf"}

def cleanup_old_data():
    """오래된 데이터 정리"""
    logger.info("오래된 데이터 정리 시작")
    time.sleep(1)
    return {"deleted_files": 25, "freed_space_mb": 500}

# 스케줄링 시스템 데모
def demo_scheduling_system():
    """스케줄링 시스템 데모"""
    
    print("🚀 스케줄링 시스템 데모 시작\n")
    
    # 1. 시스템 초기화
    print("[1/5] 스케줄링 시스템 초기화...")
    scheduler = SchedulingSystem()
    
    # 2. 작업 등록
    print("[2/5] 작업 등록...")
    
    # 매 30초마다 이미지 처리
    job1 = Job(
        name="드론 이미지 처리",
        job_type=JobType.IMAGE_PROCESSING,
        function=process_drone_images,
        interval_seconds=30,
        max_retries=3
    )
    scheduler.register_job(job1)
    
    # 매시 정각 데이터베이스 백업
    job2 = Job(
        name="데이터베이스 백업",
        job_type=JobType.DATABASE_BACKUP,
        function=backup_database,
        schedule_expression="0 * * * *",  # 매시 정각
        priority=1
    )
    scheduler.register_job(job2)
    
    # 매일 오전 6시 리포트 생성
    job3 = Job(
        name="일일 리포트 생성",
        job_type=JobType.REPORT_GENERATION,
        function=generate_daily_report,
        schedule_expression="0 6 * * *",  # 매일 06:00
        priority=2
    )
    scheduler.register_job(job3)
    
    # 매주 일요일 자정 데이터 정리
    job4 = Job(
        name="주간 데이터 정리",
        job_type=JobType.DATA_CLEANUP,
        function=cleanup_old_data,
        schedule_expression="0 0 * * 0",  # 매주 일요일 00:00
        priority=3
    )
    scheduler.register_job(job4)
    
    print(f"  ✓ {len(scheduler.registered_jobs)}개 작업 등록 완료")
    
    # 3. 폴더 감시 추가
    print("[3/5] 폴더 감시 설정...")
    
    def on_new_file(file_path):
        print(f"  새 파일 감지: {file_path}")
        # 여기에 파일 처리 로직 추가
    
    # watch_id = scheduler.folder_watcher.add_watch(
    #     "./drone_images",
    #     on_new_file,
    #     ['.jpg', '.png'],
    #     recursive=True
    # )
    # print(f"  ✓ 폴더 감시 시작: ./drone_images")
    
    # 4. 시스템 상태 확인
    print("\n[4/5] 시스템 상태...")
    status = scheduler.get_status()
    print(f"  스케줄러 상태: {'실행중' if status['scheduler_running'] else '중지'}")
    print(f"  등록된 작업: {status['total_jobs']}개")
    print(f"  활성 작업: {status['active_jobs']}개")
    print(f"  대기 작업: {status['pending_jobs']}개")
    
    # 5. 테스트 실행 (짧은 시간)
    print("\n[5/5] 테스트 실행 (10초간)...")
    print("  작업이 스케줄에 따라 실행됩니다.")
    
    # 10초간 실행
    time.sleep(10)
    
    # 이력 확인
    print("\n📊 작업 실행 이력:")
    history = scheduler.history_manager.get_history(limit=5)
    for record in history:
        print(f"  - {record['job_name']}: {record['status']} "
              f"(소요시간: {record.get('duration', 0):.2f}초)")
    
    # 시스템 종료
    print("\n시스템 종료...")
    scheduler.shutdown()
    
    print("\n✅ 스케줄링 시스템 데모 완료!")
    
    return scheduler

# 데모 실행
demo_scheduler = demo_scheduling_system()

## 7. 설정 파일 예제

In [ ]:
# 설정 파일 예제 생성
config_example = """
# 스케줄링 시스템 설정
system:
  name: "드론 작물 모니터링 스케줄러"
  timezone: "Asia/Seoul"
  
# 작업 정의
jobs:
  - name: "아침 드론 영상 처리"
    type: "image_processing"
    schedule: "0 7 * * *"  # 매일 07:00
    enabled: true
    
  - name: "점심 드론 영상 처리"
    type: "image_processing"
    schedule: "0 12 * * *"  # 매일 12:00
    enabled: true
    
  - name: "저녁 드론 영상 처리"
    type: "image_processing"
    schedule: "0 18 * * *"  # 매일 18:00
    enabled: true
    
  - name: "일일 백업"
    type: "database_backup"
    schedule: "0 2 * * *"  # 매일 02:00
    enabled: true
    
  - name: "주간 리포트"
    type: "report_generation"
    schedule: "0 9 * * 1"  # 매주 월요일 09:00
    enabled: true
    
  - name: "월간 데이터 정리"
    type: "data_cleanup"
    schedule: "0 3 1 * *"  # 매월 1일 03:00
    enabled: true

# 폴더 감시
folder_watches:
  - path: "/drone/incoming"
    extensions: [".jpg", ".jpeg", ".png", ".tiff"]
    recursive: true
    
  - path: "/drone/videos"
    extensions: [".mp4", ".avi", ".mov"]
    recursive: false

# 알림 설정
notifications:
  email:
    enabled: false
    smtp_server: "smtp.gmail.com"
    smtp_port: 587
    from_email: "drone@example.com"
    to_emails:
      - "admin@example.com"
      - "operator@example.com"
  
  slack:
    enabled: false
    webhook_url: "https://hooks.slack.com/services/..."
"""

# 설정 파일 저장
config_path = Path("scheduling_config.yaml")
with open(config_path, 'w') as f:
    f.write(config_example)

print("📝 설정 파일 예제:")
print(config_example)
print(f"\n설정 파일 저장: {config_path}")

## 8. 모니터링 대시보드

In [ ]:
class MonitoringDashboard:
    """모니터링 대시보드"""
    
    def __init__(self, scheduler: SchedulingSystem):
        self.scheduler = scheduler
        
    def get_dashboard_data(self) -> Dict:
        """대시보드 데이터 생성"""
        status = self.scheduler.get_status()
        
        # 작업 목록
        jobs = []
        for job_id, job in self.scheduler.registered_jobs.items():
            next_run = None
            scheduled_job = self.scheduler.scheduler.get_job(job_id)
            if scheduled_job:
                next_run = scheduled_job.next_run_time
            
            jobs.append({
                'id': job.id,
                'name': job.name,
                'type': job.job_type.value,
                'enabled': job.enabled,
                'next_run': next_run.isoformat() if next_run else None
            })
        
        # 최근 알림
        recent_alerts = [
            {
                'level': alert.level.value,
                'title': alert.title,
                'message': alert.message,
                'timestamp': alert.timestamp.isoformat()
            }
            for alert in self.scheduler.notification_service.get_recent_alerts(5)
        ]
        
        # 작업 통계
        stats = self.scheduler.history_manager.get_statistics(7)
        
        return {
            'system_status': status,
            'jobs': jobs,
            'recent_alerts': recent_alerts,
            'statistics': stats,
            'timestamp': datetime.now().isoformat()
        }
    
    def print_dashboard(self):
        """콘솔 대시보드 출력"""
        data = self.get_dashboard_data()
        
        print("\n" + "="*60)
        print("📊 스케줄링 시스템 대시보드")
        print("="*60)
        
        print(f"\n⚙️ 시스템 상태")
        print(f"  스케줄러: {'🟢 실행중' if data['system_status']['scheduler_running'] else '🔴 중지'}")
        print(f"  등록 작업: {data['system_status']['total_jobs']}개")
        print(f"  감시 폴더: {data['system_status']['watched_folders']}개")
        
        print(f"\n📋 예정 작업")
        for job in data['jobs'][:5]:
            status = '✅' if job['enabled'] else '⏸️'
            print(f"  {status} {job['name']} ({job['type']})")
            if job['next_run']:
                print(f"     다음 실행: {job['next_run']}")
        
        print(f"\n📈 최근 7일 통계")
        stats = data['statistics']
        if stats:
            print(f"  총 실행: {stats.get('total', 0)}회")
            print(f"  성공: {stats.get('completed', 0)}회")
            print(f"  실패: {stats.get('failed', 0)}회")
            if stats.get('avg_duration'):
                print(f"  평균 시간: {stats['avg_duration']:.2f}초")
        
        if data['recent_alerts']:
            print(f"\n🔔 최근 알림")
            for alert in data['recent_alerts'][:3]:
                icon = {'info': 'ℹ️', 'warning': '⚠️', 'error': '❌', 'critical': '🚨'}
                print(f"  {icon.get(alert['level'], '•')} {alert['title']}")
        
        print(f"\n마지막 업데이트: {data['timestamp']}")
        print("="*60)

# 대시보드 출력 예제
if 'demo_scheduler' in locals():
    print("\n대시보드 예제:")
    dashboard = MonitoringDashboard(demo_scheduler)
    # dashboard.print_dashboard()

## 9. 다음 모듈 연동

In [ ]:
print("\n🔗 다음 모듈 연동 정보\n")

print("스케줄링 시스템이 완성되었습니다!\n")

print("이 모듈이 연동하는 다른 모듈들:\n")

print("1. → Todo 2 (입력 처리):")
print("   - 폴더 감시를 통한 자동 입력 처리")
print("   - 스케줄에 따른 주기적 처리")

print("\n2. → Todo 5 (배치 처리):")
print("   - 정기적 배치 작업 실행")
print("   - 리소스 기반 작업 조절")

print("\n3. → Todo 6 (결과 저장):")
print("   - 정기적 데이터베이스 백업")
print("   - 오래된 데이터 자동 정리")

print("\n4. → Todo 8 (시각화/리포트):")
print("   - 정기 리포트 자동 생성")
print("   - 대시보드 데이터 제공")

print("\n주요 특징:")
print("✓ 크론 표현식 지원")
print("✓ 폴더 실시간 모니터링")
print("✓ 작업 실행 이력 관리")
print("✓ 알림 시스템")
print("✓ 자동 재시도 및 복구")
print("✓ 모니터링 대시보드")

print("\n지속적 모니터링 스케줄링 시스템 개발 완료! ✨")